In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [ ]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [ ]:
def mexico_michoacan(estados):

    michoacan = pd.DataFrame()
    pdfs = []
    descripciones = []
    links = []
    titulos = []
    fechas = []

    pdf_proyecto = ''
    descripcion_proyecto = ''

    ### Por paginación

    for estado in estados:
        parser_proyectos = parser(estado)

        #Link

        links_proyectos = parser_proyectos.xpath('//a[@class="image"]/@href')
        
        # titulos
        titulos_proyectos= parser_proyectos.xpath('//h2/text()')[1::]

        # Fechas
        fechas_brutas_proyectos = parser_proyectos.xpath('//p[@class="text-center"]/text()')

        fechas_brutas = []
        for f in fechas_brutas_proyectos:
            fechas_brutas.append(f.strip())

        links += links_proyectos
        titulos += titulos_proyectos
        fechas += fechas_brutas


    #### Proyecto por proyecto
    #pdfs
    #descripción

    for link in links:
        
        try:
            parser_proyectos = parser(link)

            #pdfs
            pdf_proyecto_bruto = parser_proyectos.xpath('//p//a//@href')
            pdf_proyecto = ''
            for p in pdf_proyecto_bruto:
                if '.pdf' in p:
                    pdf_proyecto = pdf_proyecto + p + ', '
            pdf_proyecto = pdf_proyecto.strip(', ')

            #Descripcion
            descripcion_proyecto_bruto = parser_proyectos.xpath('//div[@class="description"]//p/text()')
            descripcion_proyecto = ''
            for desc in descripcion_proyecto_bruto:
                desc = desc.strip()
                descripcion_proyecto = descripcion_proyecto + desc + ' '

            pdfs.append(pdf_proyecto)
            descripciones.append(descripcion_proyecto.strip())
        except:
            pdfs.append(pdf_proyecto)
            descripciones.append(descripcion_proyecto)

    michoacan['Link'] = links
    michoacan['Titulo'] = titulos
    michoacan['Fecha Cierre'] = fechas
    michoacan['Descripción'] = descripciones
    michoacan['Pdfs'] = pdfs

    return michoacan


In [ ]:
estados = ['https://icti.michoacan.gob.mx/categoria/convocatorias/page/2/', 'https://icti.michoacan.gob.mx/categoria/convocatorias/', 'https://icti.michoacan.gob.mx/categoria/convocatorias/page/3/','https://icti.michoacan.gob.mx/categoria/convocatorias/page/4/', 'https://icti.michoacan.gob.mx/categoria/convocatorias/page/5/']


michoacan = mexico_michoacan(estados)
michoacan.to_excel('Mexico_Michoacan.xlsx')